In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import math
import warnings
import joblib
import plotly.express as px
from google.colab import drive
from datetime import date, timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, CatBoostRegressor
# from plot_metric.functions import BinaryClassification

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 500

from google.colab import auth
import gspread
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [3]:
repeated_inv = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/depletion_model/repeated_inv.pkl")
time_model = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/depletion_model/time_model.pkl")
depletion_model = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/depletion_model/depletion_model.pkl")
sales_probability_model = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/depletion_model/sales_probability_model.pkl")

In [4]:
repeated_inv.sample()

,sku_number,brand,product_category,product_subcategory,seller_name,time,priority,total_units_prev,depletion,shelf_life_remaining_days,warehouse_country,listing_condition,liquidation_channel,persona_seller_type,depletion_percent
442984,G3754300,garnier,skin_care,serums_&_treatments,l'oreal_malaysia,NaN,p3,11.0,0.0,724.0,malaysia,excess,disposal,principal,0.0


In [66]:
repeated_inv[repeated_inv.sku_number == '68597735'].sample()

,sku_number,brand,product_category,product_subcategory,seller_name,time,priority,total_units_prev,depletion,shelf_life_remaining_days,warehouse_country,listing_condition,liquidation_channel,persona_seller_type,depletion_percent
151135,68597735,vaseline,skin_care,body_lotions_&_creams,unilever_indonesia,1.629419,p1,1206.0,24.12,7368.271891,indonesia,excess,distributor,principal,2.0


## Time model (default 1)

In [9]:
# ['G4079900', "l'oreal_paris", 'skin_care', 'cleansers_&_toners', "l'oreal_thailand",	600, 50, 'excess', 'liquidator']
# ['G3836500', "maybelline", 'makeup', 'face_makeup', "l'oreal_malaysia",	600, 70, 'excess', 'liquidator']
# ['G3989900', 'maybelline', 'makeup', 'face_makeup', "l'oreal_thailand",	400, 10, 'excess', 'distributor']

In [78]:
# order of test_data = ['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'shelf_life_remaining_days', 'depletion_percent', 'listing_condition', 'liquidation_channel']
test_data_point = ['67986813',	"pond's",	'skin_care',	"masks_&_exfoliators", "unilever_indonesia", 200, 50, 'excess', 'liquidator']
ans = np.expm1(time_model.predict(test_data_point))

if ans <= 5:
  ans = np.random.uniform(low=5, high=10, size=(1,))[0]
if ans >= 120:  # (smooth if more than 4 months)
  ans = np.random.uniform(low=100, high=120, size=(1,))[0]
print(ans)

60.128475536009184


In [10]:
sales_probability_model

## Sales probability model (default 2)

In [35]:
# ['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'time', 'total_units', 'shelf_life_remaining_days',
#               'listing_condition', 'liquidation_channel', 'depletion_percent']

test_data_point = ['G4246600', "l'oreal_paris", 'hair_care', 'hair_color',
       "l'oreal_thailand", 60.0, 500.0, 'excess', 'retailer',
       40]
ans = sales_probability_model.predict_proba(test_data_point)[1] * 100

if ans <= 5:
  ans = np.random.uniform(low=0, high=10, size=(1,))[0]
if ans >= 95:
   ans = np.random.uniform(low=90, high=95, size=(1,))[0]
print(ans)

15.792074398757435


## Depletion model (default 5)

In [21]:
# ['G3864100', "maybelline", 'makeup', 'lip_makeup', "l'oreal_malaysia",	600, 90, 'excess', 'liquidator']
# ['62685401', 'rinso', 'home_care', 'laundry','unilever_indonesia', 400, 90,'excess', 'retailer']
# ['G3697100', "l'oreal_paris", 'makeup', 'lip_makeup',"l'oreal_malaysia", 600, 90,'excess', 'liquidator']

In [22]:
# ['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'shelf_life_remaining_days', 'time', 'listing_condition', 'liquidation_channel']
test_data_point = ['68878865', 'pepsodent', "oral_care", 'tooth_brush', "unilever_indonesia",	100, 60, 'excess', 'liquidator']
ans = np.expm1(depletion_model.predict(test_data_point))

if ans <= 5:
  ans = np.random.uniform(low=0, high=10, size=(1,))[0]
if ans >= 95:
   ans = np.random.uniform(low=90, high=95, size=(1,))[0]
print(ans)

22.80656908186266
